<a href="https://colab.research.google.com/github/DiegoSReco/intro_python_para_economistas/blob/main/script_7_intro_encuestas_complejas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Introducción al análisis de encuestas de diseño complejo **


Los gobiernos, las empresas, ONGs, partidos políticos e investigadores necesitamos información de características, opiniones, preferencias y comportamiento de una población.

- ¿Cuántas personas están desempleadas?

- ¿Cuántas personas tienen acceso a eduación?

- ¿Por qué partido se inclinan las preferencias en las próximas elecciones?

- ¿Cuáles son las preferencias de los consumidores en cuanto a productos y servicios?

- ¿Qué políticas consideran los votantes como prioritarias?

Una opción para contestar estas preguntar es levantar censos:

- Restricción de tiempo y costos

- De acuerdo con el INEGI en el Censo Población y Vivienda 2020 se visitaron 55 millones de inmuebles y participaron 147 mil entrevistadores.

Por lo tanto para contestar a estas preguntas recurrimos al uso de encuestas complejas o encuestas de diseño probabilístico.



**¿Qué son las encuestas complejas?**

Las encuestas complejas tienen un diseño de muestra probabilístico, en donde los encuestados son seleccionados de forma aleatoria de tal forma que los resultados deben representar de forma precisa y eficiente a la población de interés.


descarga.svg

El muestreo probabilístico se caracteriza porque todos los individuos de la población tienen una probabilidad conocida y distinta de cero de ser seleccionados. Esto permite hacer inferencias estadísticas sobre la población.

- **Muestreo aleatorio simple**: Cada miembro de la población tiene la misma probabilidad de ser seleccionado.

$$ Probabilidad~de~ser~seleccionado = \frac{1}{1,000,000} = 0.000001 $$



Retos

- Nos enfrentamos a la heterogeneidad de la población.
-  Existen subgrupos con características diferentes, por lo tanto el muestreo aleatorio simple no siempre garantiza que todos estos subgrupos estén bien representados en la muestra y tengamos estimaciones precisas.



### **Conceptos básicos de encuestas complejas**

**Pesos o factores de expansión**

Son factores aplicados a cada observación en la muestra para corregir las probabilidades de selección y asegurar que los resultados sean representativos de la población total.

Ejemplo de cálculo de ponderadores:

Si suponemos que el muestreo fue aletorio simple la probabilidad de selección será la misma y la definimos como $p_i$, el peso o ponderador será igual al inverso de la probabilidad:
  $$ Peso Base (w_i) = \frac{1}{p_i}$$

  Población: 10,000 personas

  Muestra seleccionada: 500 personas  

$$p_i = \frac{500}{10,000} = 0.05 $$

$$\therefore~~~w_i = \frac{1}{0.05}=20$$


- En la realidad no es tan simple y se deben considerar ajustes por no respuesta, ajuste porestratificación o clusterisación, normalización de ponderadores, etc.



**Estimador puntual**

La mejor aproximación de una característica poblacional utilizando datos de una muestra.

**Error Estándar**

Que tan seguro o preciso es nuestro estimador puntual. Cuanta variación hay en determinada carácterísticas. Buscamos el menor error estándar.


**Interval de confianza**
Si repitiéramos muchas veces el mismo estudio, el 95% de los intervalos construidos incluirían al verdadero valor.

"No sabemos el valor exacto, pero sabemos que está en un rango determinado"



Bibliografía
- Klinger Angarita, R. (2024). Muestreo estadístico: métodos ásicos (1st ed.). Programa Editorial Universidad del Valle.
- Lehtonen, R. & Pahkinen E. (2004). Practical Methods for Design and Analysis of Complex Surveys
- Levy, P. S., & Lemeshow, S. (1993). Sampling of Populations: Methods and Applications.
- Zimmer, S. A., Powell, R. J., & Velásquez, I. C. (2024). Exploring Complex Survey Data Analysis Using R: A Tidy Introduction with {srvyr} and {survey}. Chapman & Hall: CRC Press.**
[Libro](https://tidy-survey-r.github.io/tidy-survey-book/index.html)


### **¿Cómo podemos analizar estas encuestas?**

*   Anteriormente sólo utilizábamos softwares como SPSS, SAS y Stata
*   Software libre y de código abierto como R
* ¿En Python podemos hacer análisis de encuestas complejas?

In [ ]:
import os
import tempfile
import pyarrow as pa
from zipfile import ZipFile
from io import BytesIO
import requests
import pandas as pd
import numpy as np

#Función 1: enoe_load()
def enoe_load(año, n_trim, tabla):
    """
    enoe_load() realiza una descarga de una tabla de la base de datos de la ENOE y la procesa en un DataFrame de Pandas.
    """


    url_gn_enoe = "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/microdatos"
    last_two_digits = str(año)[-2:]

    if (2005 <= año <= 2019) and (n_trim <= 4):
        list_url = [f"{url_gn_enoe}/{año}trim{n_trim}_csv.zip",
                    f"{tabla if tabla.islower() else tabla.lower()}t{n_trim}{last_two_digits}.csv"]
    elif año == 2020 and n_trim == 1:
        list_url = [
            f"{url_gn_enoe}/{año}trim{n_trim}_csv.zip",
            f"ENOE_{tabla}T{n_trim}{last_two_digits}.csv"]
    elif año == 2020 and n_trim == 2:
        etoe_link = "https://www.inegi.org.mx/contenidos/investigacion/etoe"
        raise Exception(f"Para este trimestre no existe versión ENOE. Prueba con ETOE 2020: {etoe_link}")

    elif (año == 2020 and n_trim == 3) or ((2021 <= año <= 2022) and (1 <= n_trim <= 4)):
        list_url = [
            f"{url_gn_enoe}/enoe_n_{año}_trim{n_trim}_csv.zip",
            f"ENOEN_{tabla}T{n_trim}{last_two_digits}.csv"]
    elif año == 2020 and n_trim == 4:
        list_url = [
            f"{url_gn_enoe}/enoe_n_{año}_trim{n_trim}_csv.zip",
            f"enoen_{tabla}T{n_trim}{last_two_digits}.csv"]
    elif año >= 2023 and n_trim >= 1:
        list_url = [
            f"{url_gn_enoe}/enoe_{año}_trim{n_trim}_csv.zip",
            f"ENOE_{tabla if tabla.isupper() else tabla.upper()}T{n_trim}{last_two_digits}.csv"]
    else:
        print("URL incorrecto")
        return None
    url_data = list_url[0]
    modulo = list_url[1]
    #Realizo la consulta
    request_data = requests.get(url_data).content
    #Guardo en zip file
    zip_form = ZipFile(BytesIO(request_data))
    #Cargamos datos
    with zip_form.open(modulo) as file:
         df_enoe = pd.read_csv(file, engine = "pyarrow" ,  encoding='latin-1') # na_values= [pd.NA, " "]

    return df_enoe

Cargamos ENOE y tratamos los datos

In [ ]:
#Cargamos
df_enoe_sdem = enoe_load(2024, 4 , "SDEM")

#Filtramos personas con entrevista completa y que son residentes habituales en las viviendas
df_enoe_sdem = df_enoe_sdem[(df_enoe_sdem['r_def'] ==0) & ((df_enoe_sdem['c_res']==1) | (df_enoe_sdem['c_res']==3))]

#Permanecen variables de interés
df_enoe_sdem= df_enoe_sdem[[ 'cd_a', 'r_def',  'ent','con','v_sel','tipo','mes_cal', 'n_hog','h_mud','n_ren',  't_loc_tri', 'fac_tri', 'clase1', 'clase2', 'sex','emp_ppal', 'eda', 'scian', 'salario' ,'hrsocup', 'ingocup']]

#Variable edad a numérico
df_enoe_sdem['eda'] = df_enoe_sdem['eda'].replace(' ', np.nan)
df_enoe_sdem['eda']  = pd.to_numeric(df_enoe_sdem['eda'])

#Filtramos en edad de trabajar
df_enoe_sdem = df_enoe_sdem[  (df_enoe_sdem["eda"] >= 15) & (df_enoe_sdem["eda"] < 99)  ]

#Ocupado
df_enoe_sdem['ocupado']= np.where((df_enoe_sdem['clase1']==1) & (df_enoe_sdem['clase2']==1) , 1,0)

#Codifiación SCIAN
scian_descripciones = { 1: "Agricultura, Ganadería, etc",2: "Minería", 3: "Generación y distribución de electricidad, suministro de agua y gas", 4: "Construcción",
                        5: "Industrias manufactureras", 6: "Comercio al por mayor", 7: "Comercio al por menor", 8: "Transportes, correos y almacenamiento",
                        9: "Información en medios masivos", 10: "Servicios financieros y de seguros", 11: "Servicios inmobiliarios de alquiler de bienes",
                        12: "Servicios profesionales, científicos y técnicos",13: "Corporativos", 14: "Servicios de apoyo a los negocios y manejo de desechos",15: "Servicios educativos",
                        16: "Servicios de salud y de asistencia social", 17: "Servicios de esparcimiento culturales y deportivos", 18: "Servicios de hospedaje y preparación de alimentos y bebidas",
                        19: "Otros servicios, excepto actividades gubernamentales", 20: "Actividades gubernamentales y de organismos internacionales",21: "No especificado"}
#Codificamos categorías
df_enoe_sdem['SCIAN_b'] = df_enoe_sdem['scian'].map(scian_descripciones)

#Codificación entidad con diccionario Entidades
entidad_dict = { 1:"AGS", 2: "BC", 3: "BCS", 4: "CAMP", 5: "COAH", 6: "COL", 7: "CHIS", 8: "CHIH",
                  9: "CDMX", 10: "DGO", 11: "GTO", 12: "GRO", 13: "HGO", 14: "JAL", 15: "MEX", 16: "MICH",
                  17: "MOR", 18: "NAY", 19: "NL", 20: "OAX", 21: "PUE", 22: "QRO", 23: "QR", 24: "SLP",
                  25: "SIN", 26: "SON", 27: "TAB", 28: "TAMS", 29: "TLAX", 30: "VER", 31: "YUC", 32: "ZAC"}
#Entidad con nombre abreviados
df_enoe_sdem["entidad_ab"] = df_enoe_sdem["ent"].map( entidad_dict)
#Sexo
df_enoe_sdem["clas_sexo"] = df_enoe_sdem["sex"].map({"1": "Hombre", "2": "Mujer"}).fillna(np.nan)
#Identificación de informales
df_enoe_sdem['informal'] = df_enoe_sdem['emp_ppal'].map({1:1, 2:0}).fillna(np.nan)


<ipython-input-5-e4c18f1beb3c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enoe_sdem['ocupado']= np.where((df_enoe_sdem['clase1']==1) & (df_enoe_sdem['clase2']==1) , 1,0)
<ipython-input-5-e4c18f1beb3c>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_enoe_sdem['SCIAN_b'] = df_enoe_sdem['scian'].map(scian_descripciones)
<ipython-input-5-e4c18f1beb3c>:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value ins

## **Totales y sumas**


De acuerdo con el INEGI para el Trimestre IV de 2024 la población ocupada fue de **59,487,546** mientras que la desocupada fue de **1,567,775**.

Fuente: [Boletín ENOE Trimestre IV 2024](https://www.inegi.org.mx/contenidos/saladeprensa/boletines/2025/iooe/ioe2025_01_29.pdf)

Ejemplo:

- ¿Estimar la población ocupada y desocupada en Python?




In [ ]:
#Filtro
df_pea = df_enoe_sdem[df_enoe_sdem['clase1'] == 1]
#Conteo muestral
pea_samp = df_pea.groupby('ocupado').size()

print(pea_samp)


ocupado
0      5054
1    191738
dtype: int64


In [ ]:
#Utilizamos el factor
pea_pob = df_pea.groupby('ocupado')['fac_tri'].sum()
print(pea_pob)

ocupado
0     1567775
1    59487546
Name: fac_tri, dtype: int64


**Por categorías**

Podremos hacer de dos formas:

* De acuerdo con el INEGI para el IV trimestre de 2024 los hombres ocupados fueron **34,924,786**, mientras que las mujeres **24,562,760**.


In [ ]:
pea_sexo = df_pea.groupby(['ocupado', 'clas_sexo']).size()

print(pea_sexo)

ocupado  clas_sexo
0        Hombre         2909
         Mujer          2145
1        Hombre       111371
         Mujer         80367
dtype: int64


In [ ]:

pea_sexo_pob = df_pea.groupby(['ocupado', 'clas_sexo'])['fac_tri'].sum()

print(pea_sexo_pob)

ocupado  clas_sexo
0        Hombre         942762
         Mujer          625013
1        Hombre       34924786
         Mujer        24562760
Name: fac_tri, dtype: int64


Utilizando **`crosstab()`**

In [ ]:
pea_sexo_pob_c = pd.crosstab(
                             index =   df_pea["clas_sexo"],
                             columns = df_pea['ocupado'],
                             values =  df_pea['fac_tri'],
                             aggfunc = np.sum
                             )
pea_sexo_pob_c

<ipython-input-10-14e2738cac6a>:1: FutureWarning: The provided callable <function sum at 0x7dd90d100ea0> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pea_sexo_pob_c = pd.crosstab(


ocupado,0,1
clas_sexo,,
Hombre,942762,34924786
Mujer,625013,24562760


**Medias**

Ejemplo:

- Horas e Ingresos laborales

In [ ]:
mean_ing = df_pea['ingocup'].mean()
print(f"Mean of hrs: {mean_ing:.2f}")

Mean of hrs: 6536.72


In [ ]:
mean_ing_weighted = np.average(df_pea['ingocup'], weights=df_pea['fac_tri'])

print(f"Mean of hrs: {mean_ing_weighted:.2f}")

Mean of hrs: 5878.85


In [ ]:
mean_hrs = df_pea['hrsocup'].mean()
print(f"Mean of hrs: {mean_hrs:.2f}")

mean_hrs_weighted = np.average(df_pea['hrsocup'], weights=df_pea['fac_tri'])

print(f"Mean of hrs: {mean_hrs_weighted:.2f}")


Mean of hrs: 39.82
Mean of hrs: 39.92


**Medias por categoría**



In [ ]:
df_pea.groupby('clas_sexo')['hrsocup'].mean()

,hrsocup
clas_sexo,
Hombre,42.702634
Mujer,35.826619


In [38]:
# Función para media ponderada
def mean_fac(df, grupo, variable, peso):
    media_pond = (
        df.groupby(grupo)
        .apply(lambda x: np.average(x[variable], weights=x[peso]))
        .reset_index(name=f"media_ponderada_{variable}")
        .sort_values(by=f"media_ponderada_{variable}", ascending=False))
    return media_pond


#Media de horas de trabajo semanales con factor de expansión
mean_fac_hrs = mean_fac(df_pea, 'clas_sexo', 'hrsocup', 'fac_tri')
print(mean_fac_hrs )

  clas_sexo  media_ponderada_hrsocup
0    Hombre                42.656709
1     Mujer                36.011980


<ipython-input-38-eb0ce0fc96b0>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: np.average(x[variable], weights=x[peso]))


**Media de ingreso por entidad**

- Muestra

In [35]:
mean_ing_ent = df_pea.groupby(['entidad_ab'])['ingocup'].mean()
print(mean_ing_ent.sort_values(ascending=False))

entidad_ab
BCS     11248.183496
COAH     9302.951177
NL       9021.340009
NAY      8640.191848
CAMP     8426.552030
SIN      8404.391207
TAMS     8301.509170
CHIH     8113.179722
BC       7541.163441
DGO      7526.063603
QR       7508.612503
YUC      7427.056832
COL      7320.369732
SON      7214.202633
TAB      6721.315068
MICH     6545.686072
CHIS     6151.584014
HGO      6091.719073
ZAC      5777.712774
JAL      5631.144724
VER      5630.959885
CDMX     5295.986194
TLAX     5245.635257
SLP      5213.350890
GRO      5159.107584
AGS      5079.154139
GTO      4929.332695
PUE      4909.819284
OAX      4679.951872
QRO      3792.460939
MEX      3682.525633
MOR      2724.720082
Name: ingocup, dtype: float64


- Estimación de la media poblacional del ingreso por entidad


In [39]:
#Media de ingreso con factor
mean_ing_fac = mean_fac(df_pea, 'entidad_ab', 'ingocup', 'fac_tri')

print(mean_ing_fac)

   entidad_ab  media_ponderada_ingocup
2         BCS             12108.436600
7        COAH              9225.943050
18         NL              9219.141596
17        NAY              8494.613801
27       TAMS              8376.815570
23        SIN              8193.201025
21         QR              8097.351299
5        CHIH              7905.272253
3        CAMP              7806.201668
1          BC              7522.870168
30        YUC              7375.180536
8         COL              7310.291629
9         DGO              7254.440798
25        SON              7042.774815
15       MICH              6811.637120
26        TAB              6329.682310
4        CDMX              5935.661420
13        JAL              5804.062701
12        HGO              5665.976571
31        ZAC              5472.851093
0         AGS              5337.703676
28       TLAX              5283.603785
29        VER              5276.281985
24        SLP              5215.467494
6        CHIS            

<ipython-input-38-eb0ce0fc96b0>:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: np.average(x[variable], weights=x[peso]))



##✅ **¿Qué sí se puede hacer en Python?**
- Calcular estadísticos descriptivos ponderados (medias, totales) usando los pesos.

**Limitaciones**

En Python, no se pueden calcular de forma directa y automatizada los errores estándar ni los intervalos de confianza ajustados por el diseño complejo de encuestas (es decir, considerando estratificación, conglomerados y pesos simultáneamente), como sí se puede en R con el paquete survey.
